In [ ]:
import cv2 # computer vision :for various image and video processing tasks.
# It's commonly used for tasks like image manipulation, object detection, face recognition, and more.

from tensorflow.keras.models import Sequential # essential for building a sequential neural network model in Keras.

from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense
# Conv2D is used for convolutional operations, which are fundamental in CNNs for extracting features from images.
# MaxPooling2D is used for downsampling the spatial dimensions of the input.It helps reduce the computational complexity and control overfitting by retaining the most important information.
# Dropout used to prevent overfitting by randomly setting a fraction of input units to 0 at each update during training.
# Flatten is used to reshape the data from a multi-dimensional tensor into a 1D array
# Dense represents a fully connected layer, where each neuron is connected to all neurons

from tensorflow.keras.optimizers import Adam # build and compile a neural network model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#generating batches of augmented image data,which is commonly used for training deep learning models.
# particularly for computer vision tasks like image classification and object detection.

import numpy as np

In [ ]:
# unzip the dataset
!unzip -uq "/content/drive/MyDrive/face mask/archive1-220321-141335.zip" -d "/content/drive/MyDrive/face mask/archive"

In [ ]:
#store each paths to corresponding variables for easy usage
import os
main_dir="/content/drive/MyDrive/face mask/archive/New Masks Dataset"
train_dir=os.path.join(main_dir,'Train')
test_dir=os.path.join(main_dir,'Test')
valid_dir=os.path.join(main_dir,'Validation')

train_mask_dir=os.path.join(train_dir,'Mask')
train_nomask_dir=os.path.join(train_dir,'Non Mask')
test_mask_dir=os.path.join(test_dir,'Mask')
test_nomask_dir=os.path.join(test_dir,'Non Mask')
valid_mask_dir=os.path.join(valid_dir,'Mask')
valid_nomask_dir=os.path.join(valid_dir,'Non Mask')

valid_nomask_dir
test_nomask_dir

'/content/drive/MyDrive/face mask/archive/New Masks Dataset/Test/Non Mask'

In [ ]:
#optional
train_mask_names=os.listdir(train_mask_dir)
train_nomask_names=os.listdir(train_nomask_dir)
train_nomask_names[1:10]

['1.jpg',
 '10.jpg',
 '100.jpg',
 '101.jpg',
 '102.jpg',
 '104.jpg',
 '105.jpg',
 '106.jpg',
 '107.jpg']

In [ ]:

train_datagen= ImageDataGenerator(rescale=1.0/255, #brings the pixel values into the range of [0, 1]
                                  zoom_range=0.2,
                                  rotation_range=40,
                                  horizontal_flip = True)
test_datagen=ImageDataGenerator(rescale=1.0/255)
validation_datagen=ImageDataGenerator(rescale=1.0/255)

train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size=(150,150),#(width,height)
                                                  batch_size=32, #no of images in each batch
                                                  class_mode='binary') #class_index
test_generator=test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150,150),
                                                  batch_size=32,
                                                  class_mode='binary')
valid_generator=validation_datagen.flow_from_directory(valid_dir,
                                                  target_size=(150,150),
                                                  batch_size=32,
                                                  class_mode='binary') #two types : binary,categorical

Found 600 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 306 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'Mask': 0, 'Non Mask': 1}

In [ ]:
train_generator.image_shape

(150, 150, 3)

In [ ]:


# Create a Sequential model
model = Sequential()

# Add layers to the model
model.add(Conv2D(32, (3, 3), padding='SAME', activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3, 3), padding='SAME', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Binary classification, so using sigmoid activation

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 75, 75, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 37, 37, 64)       

In [ ]:
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,
                  epochs=30,
                  validation_data = valid_generator)

Epoch 1/30
19/19 [==============================] - 65s 3s/step - loss: 1.0692 - accuracy: 0.5700 - val_loss: 0.6893 - val_accuracy: 0.6242
Epoch 2/30
19/19 [==============================] - 54s 3s/step - loss: 0.4852 - accuracy: 0.8150 - val_loss: 0.4475 - val_accuracy: 0.8529
Epoch 3/30
19/19 [==============================] - 66s 4s/step - loss: 0.3913 - accuracy: 0.8500 - val_loss: 0.4713 - val_accuracy: 0.8693
Epoch 4/30
19/19 [==============================] - 52s 3s/step - loss: 0.2914 - accuracy: 0.9033 - val_loss: 0.3744 - val_accuracy: 0.8954
Epoch 5/30
19/19 [==============================] - 51s 3s/step - loss: 0.2578 - accuracy: 0.9100 - val_loss: 0.3309 - val_accuracy: 0.8954
Epoch 6/30
19/19 [==============================] - 53s 3s/step - loss: 0.2436 - accuracy: 0.9150 - val_loss: 0.3474 - val_accuracy: 0.8987
Epoch 7/30
19/19 [==============================] - 62s 3s/step - loss: 0.2353 - accuracy: 0.9167 - val_loss: 0.2951 - val_accuracy: 0.9020
Epoch 8/30
19/19 [==

In [ ]:
test_loss,test_acc=model.evaluate(test_generator)
print('test acc :{} test loss : {}'.format(test_acc,test_loss))

4/4 [==============================] - 3s 583ms/step - loss: 1.1381 - accuracy: 0.5000
test acc :0.5 test loss : 1.1380717754364014


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()
for f in uploaded.keys():
  image_path='/content'+f
  img=image.load_img(image_path,target_size=(150,150))
  images=image.img_to_array(img)
  #images=np.expand_dims(images,axis=0)
  prediction=model.predict(images)
  if(prediction==0):
    print(f,"mask is there")
  else:
    print(f,"no mask is present")


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing.image import load_img

# Assuming 'model' is already defined and loaded with your trained model

uploaded = files.upload()
for f in uploaded.keys():
    image_path = '/content/' + f
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)  # Changed 'images' to 'img_array'
    img_array = np.expand_dims(img_array, axis=0)  # Expanding dimensions

    prediction = model.predict(img_array)
    if prediction == 0:
        print(f, "mask is there")
    else:
        print(f, "no mask is present")


ImportError: ignored

In [ ]:
model.save('saved_model07.h5')